##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar o TensorBoard: o kit de ferramentas de visualização do TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tensorboard"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tensorboard.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>


O [TensorBoard](https://www.tensorflow.org/tensorboard) é uma ferramenta integrada para fornecer medições e visualizações no TensorFlow. Métricas comuns de experimentos de aprendizado de máquina, como precisão e perdas, podem ser rastreadas e exibidas no TensorBoard. O TensorBoard é compatível com o código do TensorFlow 1 e 2.

No TensorFlow 1, o `tf.estimator.Estimator` salva resumos para TensorBoard por padrão. Em comparação, no TensorFlow 2, os resumos podem ser salvos usando um <a>callback</a> <code>tf.keras.callbacks.TensorBoard</code>.

Este guia demonstra como usar o TensorBoard, primeiro, no TensorFlow 1 com Estimators e, em seguida, como realizar o processo equivalente no TensorFlow 2.

### Configuração

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import tempfile
import numpy as np
import datetime
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist # The MNIST dataset.

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### TensorFlow 1: TensorBoard com tf.estimator

Neste exemplo do TensorFlow 1, você instancia um `tf.estimator.DNNClassifier`, faz seu treinamento e avaliação no dataset MNIST e usa o TensorBoard para exibir as métricas:

In [ ]:
%reload_ext tensorboard

feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.1,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%tensorboard --logdir {classifier.model_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF1.png"/> -->

### TensorFlow 2: TensorBoard com callback Keras e Model.fit

Neste exemplo com TensorFlow 2, você cria e armazena logs com o callback `tf.keras.callbacks.TensorBoard` e treina o modelo. O callback rastreia a exatidão e a perda por época. Ele é passado para `Model.fit` na lista `callbacks`.

In [ ]:
%reload_ext tensorboard

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=1) # Enable histogram computation with each epoch.

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir {tensorboard_callback.log_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF2.png"/> -->


# Próximos passos

- Saiba mais sobre o TensorBoard no Guia [Primeiros passos](https://www.tensorflow.org/tensorboard/get_started).
- Para APIs de nível inferior, consulte o guia [Migração do tf.summary para oTensorFlow 2](https://www.tensorflow.org/tensorboard/migrate).